In [1]:
from scipy import optimize
import matplotlib.pyplot as plt
import numpy as np
#import autograd.numpy as numpy
#from autograd import grad
#from sympy import diff
from math import sqrt
from math import tanh
import scipy

In [27]:
def numerical_gradient(f,x):
    #f:function x:ndarray
    h=1e-10 #∆x
    grad=np.zeros_like(x)

    for index in range(x.size):
        temp_val=x[index]
        x[index]=temp_val+h
        fxh1=f(x)
        x[index]=temp_val-h
        fxh2=f(x)

        grad[index]=(fxh1-fxh2)/(2*h)
        x[index]=temp_val

    return  grad

In [28]:
def numerical_gradient_param(f,x,param):
    #f:function x:ndarray
    h=1e-10 #∆x
    grad=np.zeros_like(x)

    for index in range(x.size):
        temp_val=x[index]
        x[index]=temp_val+h
        fxh1=f(x,param)
        x[index]=temp_val-h
        fxh2=f(x,param)

        grad[index]=(fxh1-fxh2)/(2*h)
        x[index]=temp_val

    return  grad

In [29]:
Nu, Ks, Kb, epsilon, sigma, Tamb, W, L, h, Ts = 10, 400, 0.0267, 0.35, 5.67E-8, 293.15, 0.2, 0.2, 0.02, 373.15
# X[1],X[2],X[3]: H,b,s
# h:height of base
x0max = 1
x1max = 1
x2max = 1
x0min = 1e-03
x1min = 1e-03
x2min = 1e-03

In [54]:
def Rbase(X):
    return h / (Ks * W * L)


def Rconv(X):
    return 1 / (hconv(X) * S(X))


def Rrad(X):
    return 1 / (hrad(X) * S(X))


def Rfin(X):
    return (X[2] + X[1]) / (W * L * np.sqrt(np.abs(Nu * Ks * Kb * X[1] / X[2])) *
                            np.tanh(X[0] * np.sqrt(np.abs(Nu * Ks /
                                                         (Kb * X[1] * X[2])))))


def hconv(X):
    return Nu * Ks / (2 * X[2])


def hrad(X):
    return epsilon * sigma * (Ts + Tamb) * (Ts**2 + Tamb**2)


def S(X):
    return 4 * W * L + (n(X) - 1) * X[2] * (L - X[0]) - (2 * n(X) -
                                                         2) * X[0] * L


def n(X):
    return (W - X[1]) / X[2] + 1


def f(X):
    return Rbase(X) + Rfin(X) + (Rconv(X) * Rrad(X)) / (Rconv(X) + Rrad(X))

In [55]:
cons = (
    {
        'type': 'ineq',
        'fun': lambda X: X[0] - x0min
    },
    {
        'type': 'ineq',
        'fun': lambda X: X[1] - x1min
    },
    {
        'type': 'ineq',
        'fun': lambda X: X[2] - x2min
    },
    {
        'type': 'ineq',
        'fun': lambda X: -X[0] + x0max
    },
    {
        'type': 'ineq',
        'fun': lambda X: -X[1] + x1max
    },
    {
        'type': 'ineq',
        'fun': lambda X: -X[2] + x2max
    },
)

In [56]:
x_start = optimize.brute(f, ((x0min, x0max), (x1min, x1max), (x2min, x2max)),
                         Ns=10,
                         finish=None)
#枚举Ns次确认起点
print(x_start)

[0.112 0.001 0.001]


In [57]:
#X0 = [0.1, 0.1, 0.1]
#res = optimize.minimize(f,
#                        x_start,
#                        method='SLSQP',
#                        constraints=cons,
#                        options={'maxiter': 1000})
#print(res)

In [58]:
x = []
xstar=[]
x.append(np.array(x_start))
delta = 0.001
i = 0
max_iter = 10
a=0
k_max=100

In [61]:
def Pf(x, param):
    a = param[0]
    xi = param[1]
    return f(xi) + (numerical_gradient(f, x).T) @ (x - xi) + 0.5 * (
        (x - xi).T * a) @ (x - xi)


def LFOPC(x0, a, delta,k_max):
    Pfparam = [a, x0]
    x = []
    a=[]
    v=[]
    x.append(np.array(x0))
    i_x=0
    i_xm = 2
    i_s = 0
    i_sm = 3
    i_d=0
    i_dm=5
    p=1
    k=-1
    sigma_t=0.001
    epsilon_g=0.00001
    epsilon_x=0.0001
    print(x0)
    a.append(-numerical_gradient_param(Pf,x[0],Pfparam))
    delta_t=np.sqrt(delta/(5*np.linalg.norm(x[0])))
    v.append(a[0]*delta_t)
    while k<100:
        k=k+1
        delta_x_norm=np.linalg.norm(v[k])*delta_t
        if delta_x_norm>=delta:
            i_d=0
            p=p+sigma_t
            delta_t=p*delta_t
        else:
            i_d+=1
            v[k]=delta*v[k]/(delta_t*np.linalg.norm(v[k]))
        if i_s>=i_sm:
            r=1-80*epsilon_x/delta
            v[k]=(v[k]+r*v[k])/4
            x[k]=(x[k]+x[k-1])/2
            i_s=0
        x.append(x[k]+v[k]*delta_t)
        while True:
            a.append(-numerical_gradient_param(Pf,x[0],Pfparam))
            v.append(v[k]+a[k+1]*delta_t)
            if(a[k+1].T@a[k]>0):
                i_s=0
            else:
                if i_d>i_dm:
                    delta_t=delta_t/2
                    i_d=0
                i_s=i_s+1
                p=1
            if np.linalg.norm(a[k+1]<=epsilon_g):
                return x[k+1]
            if np.linalg.norm(x[k+1]-x[k])<epsilon_x:
                return x[k+1]
            if k>k_max:
                return x[k+1]
            if np.linalg.norm(v[k+1])>np.linalg.norm(v[k]):
                i_x=0
                break
            x.append((x[k+1]+x[k])/2)
            i_x=i_x+1
            if i_x<=i_xm:
                v.append((v[k+1]+v[k])/4)
                k=k+1
            else:
                v.append(np.array(0))
                k=k+1
                i_xm=1

In [62]:
while i < max_iter:
    xi=x[i]
    xi_=x[i-1]
    fxi=f(x[i])
    fxi_=f(x[i-1])
    dfxi=numerical_gradient(f,np.array(x[i]))
    if i!=0:
        a=(2*(fxi_-fxi-(dfxi.T@(xi_-xi))))/(np.linalg.norm(xi_-xi))**2
    xstar.append(LFOPC(xi,a,delta,k_max))
    i+=1
    x.append(xstar[i-1])
print(x[i])

[ 0.20139418 -0.05093922 -0.03590974]
[ 0.20139409 -0.05122369 -0.04017219]
[ 0.201394   -0.0514422  -0.04450493]
[ 0.20139388 -0.05158499 -0.04890969]
[ 0.20139376 -0.05164213 -0.05338761]
[ 0.20139362 -0.05160323 -0.05793962]
[ 0.20139346 -0.0514571  -0.06256669]
[ 0.2013933  -0.05119137 -0.06727011]


In [ ]:
#a = np.array([0.11,0.1,0.1])

#autograd
#grad_f = grad(f)
#grad_f(a).tolist()
#numpy.sqrt, numpy.tanh for autograd


#diy function
#print(f(a))
#print(numerical_gradient(f,a))

In [ ]:
help(np.T)